<a href="https://colab.research.google.com/github/JoshuaOmondi/Data-Projects/blob/master/FPL_Team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using Linear Programming to Choose the Best FPL Team**

These are the steps to be used.
* Describing the problem.( Building a team that gives maximum pts given the budget)
* Formulate the mathematical program
* Solving the mathematical program
* Evaluating the result
* Finalizing the result
Data required: player price, team, points and position.
Constraints: budget and number of players.

In [ ]:
pip install pulp

     |████████████████████████████████| 41.2 MB 77 kB/s 


In [ ]:
#importing libraries
import pandas as pd
from pulp import *

In [5]:
! git clone https://github.com/vaastav/Fantasy-Premier-League.git

Cloning into 'Fantasy-Premier-League'...
remote: Enumerating objects: 154063, done.
remote: Counting objects: 100% (32995/32995), done.
remote: Compressing objects: 100% (14456/14456), done.
remote: Total 154063 (delta 15615), reused 32168 (delta 14964), pack-reused 121068
Receiving objects: 100% (154063/154063), 55.83 MiB | 18.40 MiB/s, done.
Resolving deltas: 100% (72651/72651), done.
Checking out files: 100% (8296/8296), done.


In [6]:
#Loading data
PATH = 'Fantasy-Premier-League/data/'

current_df = pd.read_csv(PATH + '2021-22/gws/merged_gw.csv')
current_df.tail()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
2897,Wilfred Ndidi,MID,Leicester,2.3,0,0,9,0,3.2,216,42,2,0,1.9,13.8,2021-09-19T13:00:00Z,90,4,0,0,0,0,5,0,124343,1,2,2.0,1,-2718,10241,12959,50,False,1,5
2898,Matt Ritchie,DEF,Newcastle,0.3,0,0,12,0,33.3,292,46,1,0,5.7,15.2,2021-09-17T19:00:00Z,90,10,0,0,0,0,5,0,23262,1,1,8.0,1,804,4194,3390,50,True,1,5
2899,Nathan Redmond,MID,Southampton,0.8,0,0,18,1,23.9,336,45,0,0,5.5,14.8,2021-09-18T14:00:00Z,90,12,0,0,0,0,5,0,7400,0,0,16.0,3,-338,347,685,59,False,0,5
2900,Mathew Ryan,GK,Brighton,0.0,0,0,0,0,0.0,65,42,0,0,0.0,0.0,2021-09-19T13:00:00Z,0,9,0,0,0,0,5,0,808,1,2,0.0,0,-71,0,71,45,True,0,5
2901,Ryan Fredericks,DEF,West Ham,0.0,0,0,0,0,0.0,415,49,0,0,0.0,0.0,2021-09-19T13:00:00Z,0,13,0,0,0,0,5,0,10636,2,1,0.0,0,-594,474,1068,45,True,0,5


In [7]:
# Get data for GW5
gw_5 = current_df[current_df.GW == 5]
gw_5.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
2296,Eric Bailly,DEF,Man Utd,0.0,0,0,0,0,0.0,286,49,0,0,0.0,0.0,2021-09-19T13:00:00Z,0,19,0,0,0,0,5,0,7265,2,1,0.0,0,-624,252,876,50,False,0,5
2297,Keinan Davis,FWD,Aston Villa,0.0,0,0,0,0,0.0,49,41,0,0,0.0,0.0,2021-09-18T16:30:00Z,0,8,0,0,0,0,5,0,115711,0,3,0.0,0,-7359,9224,16583,45,True,0,5
2298,Ayotomiwa Dele-Bashiru,MID,Watford,0.0,0,0,0,0,0.0,394,47,0,0,0.0,0.0,2021-09-18T14:00:00Z,0,15,0,0,0,0,5,0,1709,3,1,0.0,0,-267,0,267,45,False,0,5
2299,James Ward-Prowse,MID,Southampton,3.0,0,0,13,1,12.2,341,45,0,0,2.6,13.6,2021-09-18T14:00:00Z,90,12,0,0,0,0,5,0,268540,0,0,0.0,3,-20295,12541,32836,64,False,0,5
2300,Bruno Miguel Borges Fernandes,MID,Man Utd,3.2,0,0,17,0,48.8,277,49,1,0,10.5,18.0,2021-09-19T13:00:00Z,90,19,0,0,0,0,5,0,2797372,2,1,38.0,2,-552801,55156,607957,120,False,0,5


In [8]:
data = gw_5[['name', 'team', 'position', 'total_points', 'value']]
data.head()


,name,team,position,total_points,value
2296,Eric Bailly,Man Utd,DEF,0,50
2297,Keinan Davis,Aston Villa,FWD,0,45
2298,Ayotomiwa Dele-Bashiru,Watford,MID,0,45
2299,James Ward-Prowse,Southampton,MID,3,64
2300,Bruno Miguel Borges Fernandes,Man Utd,MID,2,120


**Initializing Variables**

Our aim is to optimize the total points

In [9]:
# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()
BUDGET = 1000
pos_available = {
    'DEF': 5,
    'FWD': 3,
    'MID': 5,
    'GK': 2,
}

# Initialize Variables
names = [data.name[i] for i in data.index]
teams = [data.team[i] for i in data.index]
positions = [data.position[i] for i in data.index]
prices = [data.value[i] for i in data.index]
points = [data.total_points[i] for i in data.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in data.index]

**Initializing the Problem**

Since we intend to maximize the number of points, we use a maximization function.

In [10]:
# Initialize the problem
prob = LpProblem("FPL Player Choices", LpMaximize)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [11]:
# Define the objective
prob += lpSum(players[i] * points[i] for i in range(len(data))) 

In [12]:
# Build the constraints
prob += lpSum(players[i] * data.value[data.index[i]] for i in range(len(data))) <= BUDGET # Budget Limit

for pos in POS:
  prob += lpSum(players[i] for i in range(len(data)) if positions[i] == pos) <= pos_available[pos] # Position Limit

for club in CLUBS:
  prob += lpSum(players[i] for i in range(len(data)) if teams[i] == club) <= 3 # Club Limit

In [13]:
# Solve the problem
prob.solve()

1

In [14]:
# Rertrieving player names that adhere to the constraints
for v in prob.variables():
  if v.varValue != 0:
    name = data.name[int(v.name.split("_")[1])]
    club = data.team[int(v.name.split("_")[1])]
    position = data.position[int(v.name.split("_")[1])]
    point = data.total_points[int(v.name.split("_")[1])]
    price = data.value[int(v.name.split("_")[1])]
    print(name, position, club, point, price, sep=" | ")

Joshua King | FWD | Watford | 10 | 55
Aaron Ramsdale | GK | Arsenal | 9 | 45
Mohamed Salah | MID | Liverpool | 12 | 125
Virgil van Dijk | DEF | Liverpool | 12 | 65
Saïd Benrahma | MID | West Ham | 10 | 64
Antonio Rüdiger | DEF | Chelsea | 14 | 55
Ismaila Sarr | MID | Watford | 15 | 60
Marcos Alonso | DEF | Chelsea | 10 | 57
Matthew Cash | DEF | Aston Villa | 15 | 50
Ivan Toney | FWD | Brentford | 12 | 63
David de Gea | GK | Man Utd | 10 | 50
Danny Welbeck | FWD | Brighton | 9 | 60
Martin Ødegaard | MID | Arsenal | 11 | 55
Leon Bailey | MID | Aston Villa | 9 | 64
Thiago Emiliano da Silva | DEF | Chelsea | 15 | 54


In [15]:
#Obtaining the expected costs and total points
score = str(prob.objective)
constraint = [str(const) for const in prob.constraints.values()][0]
for v in prob.variables():
  score = score.replace(v.name, str(v.varValue))
  constraint = constraint.replace(v.name, str(v.varValue))

score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

print("Constraint: ")
print(constraint_pretty + " = " + str(eval(constraint_pretty)))
print()
print("Score: ")
print(score_pretty + " = " + str(eval(score_pretty)))

Constraint: 
55*1.0 + 45*1.0 + 125*1.0 + 65*1.0 + 64*1.0 + 55*1.0 + 60*1.0 + 57*1.0 + 50*1.0 + 63*1.0 + 50*1.0 + 60*1.0 + 55*1.0 + 64*1.0 + 54*1.0 = 922.0

Score: 
10*1.0 + 9*1.0 + 12*1.0 + 12*1.0 + 10*1.0 + 14*1.0 + 15*1.0 + 10*1.0 + 15*1.0 + 12*1.0 + 10*1.0 + 9*1.0 + 11*1.0 + 9*1.0 + 15*1.0 = 173.0
